### 1

In [5]:
!curl localhost:9200

{
  "name" : "f47178e508bc",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "vDCPI3tBQgK4N2pLBqjIeg",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


### 2

In [6]:
import requests

docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

In [7]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

In [8]:
from tqdm.auto import tqdm

index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### 3

In [11]:
query = "How do I execute a command in a running docker container?"

In [12]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields",
                }
            },
        }
    },
}

In [13]:
response = es_client.search(index=index_name, body=search_query)

In [20]:
response["hits"]["hits"][0]["_score"]

84.050095

### 4

In [21]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields",
                }
            },
            "filter": {"term": {"course": "machine-learning-zoomcamp"}},
        }
    },
}

response = es_client.search(index=index_name, body=search_query)

In [22]:
response["hits"]["hits"][2]

{'_index': 'course-questions',
 '_id': 'JnQ6a5ABy4C6IJ1FG2R4',
 '_score': 49.938507,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

### 5

In [25]:
result_docs = []

for docs in response["hits"]["hits"]:
    result_docs.append(docs["_source"])

In [45]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

for doc in result_docs:
    context += context_template.format(question=doc["question"], text=doc["text"]) + "\n\n"

context = context[:-2]

In [46]:
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [47]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [48]:
prompt = prompt_template.format(question=query, context=context)

In [49]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [50]:
len(prompt)

1462

### 6

In [52]:
# !pip install tiktoken

In [54]:
import tiktoken


encoding = tiktoken.encoding_for_model("gpt-4o")

In [57]:
encoding.encode(prompt)

[63842,
 261,
 4165,
 14029,
 29186,
 13,
 30985,
 290,
 150339,
 4122,
 402,
 290,
 31810,
 8099,
 591,
 290,
 40251,
 7862,
 558,
 8470,
 1606,
 290,
 19719,
 591,
 290,
 31810,
 8099,
 1261,
 55959,
 290,
 150339,
 364,
 107036,
 25,
 3253,
 621,
 357,
 15792,
 261,
 6348,
 306,
 261,
 6788,
 62275,
 9282,
 1715,
 10637,
 50738,
 734,
 48,
 25,
 3253,
 621,
 357,
 15199,
 261,
 62275,
 9282,
 3901,
 32,
 25,
 41281,
 290,
 9282,
 3621,
 306,
 25383,
 6766,
 326,
 151187,
 290,
 7251,
 4859,
 11,
 813,
 484,
 480,
 13217,
 261,
 38615,
 6348,
 558,
 68923,
 2461,
 533,
 278,
 2230,
 7962,
 4859,
 38615,
 464,
 3365,
 523,
 3335,
 290,
 9282,
 382,
 4279,
 6788,
 11,
 15792,
 261,
 6348,
 306,
 290,
 4857,
 9282,
 734,
 68923,
 10942,
 350,
 6555,
 290,
 9282,
 26240,
 446,
 68923,
 25398,
 533,
 278,
 464,
 6896,
 26240,
 29,
 38615,
 198,
 6103,
 277,
 10732,
 391,
 79771,
 1029,
 48,
 25,
 3253,
 621,
 357,
 5150,
 6291,
 591,
 922,
 2698,
 7342,
 316,
 62275,
 9282,
 3901,
 32,
 2

In [56]:
len(encoding.encode(prompt))

322

### Bonus: generating the answer (ungraded)

In [66]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(model="gpt-4o", messages=[{"role": "user", "content": prompt}])
answer = response.choices[0].message.content

In [67]:
print(answer)

To execute a command in a running Docker container, follow these steps:

1. Find the container ID by listing all running containers:
   ```bash
   docker ps
   ```

2. Execute a command in the specific container:
   ```bash
   docker exec -it <container-id> <command>
   ```

For example, to start a bash session in the running container, use:
```bash
docker exec -it <container-id> bash
```

Replace `<container-id>` with the actual container ID obtained from the `docker ps` command.


In [68]:
len(encoding.encode(answer))

115

In [74]:
response.usage

CompletionUsage(completion_tokens=115, prompt_tokens=329, total_tokens=444)

### Bonus: calculating the costs (ungraded)

In [75]:
input_price = 150 * 5 / 10**6
output_price = 250 * 15 / 10**6

full_price = 1000 * (input_price + output_price)
full_price

4.5

In [78]:
input_price = 322 * 5 / 10**6
output_price = 115 * 15 / 10**6

full_price = 1000 * (input_price + output_price)
full_price

3.3350000000000004